# Inference with pre-trained model
This notebook demonstrates a standardized pipeline for utilizing pre-trained models and their associated weights from the Torch Hub. The pipeline allows for the selection of a specific model and its weights, followed by inference on a provided image.

## Resources
- [Torch Vision Model](https://pytorch.org/vision/stable/models.html) : List of available models and weights, pretrain and 
- [Torch Official Tutorial](https://pytorch.org/hub/pytorch_vision_resnet/) : Tutorial on basic pytorch.


# Summary 
## One Cell scripts for pretrain model and inference
Individuals who possess the ability to comprehend and follow the scripts presented in the subsequent cell may forgo the explanatory sections, as these sections provide a more detailed analysis of the aforementioned cells.

In [7]:
import torch
import urllib
from PIL import Image
from torchvision import transforms

# Load a specify pretrain model on ImageNet
model = torch.hub.load('pytorch/vision', 'resnet18', weights="IMAGENET1K_V1")
model.eval()

# Download an example image from the pytorch website
url, filename = ("https://github.com/pytorch/hub/raw/master/images/dog.jpg", "dog.jpg")
try: urllib.URLopener().retrieve(url, filename)
except: urllib.request.urlretrieve(url, filename)

## Load the image into a PIL image
## make sure it is of certain size and normalized
input_image = Image.open(filename)
preprocess = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])
input_tensor = preprocess(input_image)
input_batch = input_tensor.unsqueeze(0) # create a mini-batch as expected by the model

# Move the input and model to GPU for speed if available
if torch.cuda.is_available():
    input_batch = input_batch.to('cuda')
    model.to('cuda')

with torch.no_grad():
    output = model(input_batch)
# Tensor of shape 1000, with confidence scores over ImageNet's 1000 classes
print(output[0].shape)
probabilities = torch.nn.functional.softmax(output[0], dim=0)

# Download ImageNet labels
!wget https://raw.githubusercontent.com/pytorch/hub/master/imagenet_classes.txt

# Read the categories
with open("imagenet_classes.txt", "r") as f:
    categories = [s.strip() for s in f.readlines()]
# Show top categories per image
top5_prob, top5_catid = torch.topk(probabilities, 5)
for i in range(top5_prob.size(0)):
    print(categories[top5_catid[i]], top5_prob[i].item())

Using cache found in /home/howt51/.cache/torch/hub/pytorch_vision_main


torch.Size([1000])
--2024-04-12 06:55:57--  https://raw.githubusercontent.com/pytorch/hub/master/imagenet_classes.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.110.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 10472 (10K) [text/plain]
Saving to: ‘imagenet_classes.txt.2’

imagenet_classes.tx 100%[===================>]  10.23K  --.-KB/s    in 0.003s  

2024-04-12 06:55:58 (3.69 MB/s) - ‘imagenet_classes.txt.2’ saved [10472/10472]

Samoyed 0.8846226930618286
Arctic fox 0.04580523073673248
white wolf 0.044276077300310135
Pomeranian 0.005621373653411865
Great Pyrenees 0.004652000498026609


# Breakdown

## Model Loading
The model is loaded using torch.hub.load() function, the first argument is the repository name, the second argument is the model name, and the third argument is the model version. 

### Torch Hub - Models

- **Pre-trained Weights**: The script utilizes the torch.hub.load() function to acquire the pre-trained "resnet18" model, leveraging its weights trained on the ImageNet dataset.
- **Evaluation Mode**: The model.eval() method is employed to transition the loaded model into evaluation mode. 
    - This mode deactivates functionalities like Dropout and BatchNorm layers, which are not necessary for the inference task and can even hinder performance..

In [8]:
import torch
model = torch.hub.load('pytorch/vision', 'resnet18', weights="IMAGENET1K_V1")
model.eval()

Using cache found in /home/howt51/.cache/torch/hub/pytorch_vision_main


ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

## Download an example image
 from the pytorch website

In [9]:
# Download an example image from the pytorch website
import urllib
url, filename = ("https://github.com/pytorch/hub/raw/master/images/dog.jpg", "dog.jpg")
try: urllib.URLopener().retrieve(url, filename)
except: urllib.request.urlretrieve(url, filename)

## Loading the Image and pretransforming
We will use torchvision and PIL to load the image and pre-transform it before feeding it to the model.

In [10]:
# Load the image into a PIL image
## make sure it is of certain size and normalized
input_image = Image.open(filename)
preprocess = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])
input_tensor = preprocess(input_image)
input_batch = input_tensor.unsqueeze(0) # create a mini-batch as expected by the model

# Move the input and model to GPU for speed if available
if torch.cuda.is_available():
    input_batch = input_batch.to('cuda')
    model.to('cuda')

with torch.no_grad():
    output = model(input_batch)
# Tensor of shape 1000, with confidence scores over ImageNet's 1000 classes
print(output[0].shape)
probabilities = torch.nn.functional.softmax(output[0], dim=0)

torch.Size([1000])


# Inference Execution
Scripts for inference
- **Gradient Calculation Deactivation**: The *torch.no_grad()* context manager is employed within the inference process to disable gradient computation. This optimization reduces memory consumption during inference, as gradients are not required for updating model weights.

- **Probability Acquisition**: A Softmax function is applied at the conclusion of the inference pipeline. This function transforms the model's output logits into probability scores, representing the likelihood of each potential class assignment for the input data.

In [11]:
with torch.no_grad():
    output = model(input_batch)
# Tensor of shape 1000, with confidence scores over ImageNet's 1000 classes
print(output[0].shape)
probabilities = torch.nn.functional.softmax(output[0], dim=0)

torch.Size([1000])


## Acquiring the Top Predictions labels

The script leverages the wget command-line utility to retrieve the ImageNet class labels from a publicly accessible source. This is likely achieved through the following steps:

1. **URL Construction**: The script constructs a URL pointing to the desired resource, in this case, the imagenet_classes.txt file hosted on a public repository like GitHub.

2. **Download Initiation**: The wget command is then executed, specifying the constructed URL as an argument. This initiates the download process, transferring the file from the remote server to the local environment.

3. **Top class acquiring**: Torch included scripts to aid in showing the top categories via *torch.topk*, along with top class.


In [12]:
# Download ImageNet labels
!wget https://raw.githubusercontent.com/pytorch/hub/master/imagenet_classes.txt

# Read the categories
with open("imagenet_classes.txt", "r") as f:
    categories = [s.strip() for s in f.readlines()]
# Show top categories per image
top5_prob, top5_catid = torch.topk(probabilities, 5)
for i in range(top5_prob.size(0)):
    print(categories[top5_catid[i]], top5_prob[i].item())

--2024-04-12 06:56:00--  https://raw.githubusercontent.com/pytorch/hub/master/imagenet_classes.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.110.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 10472 (10K) [text/plain]
Saving to: ‘imagenet_classes.txt.3’

imagenet_classes.tx 100%[===================>]  10.23K  --.-KB/s    in 0s      

2024-04-12 06:56:01 (90.1 MB/s) - ‘imagenet_classes.txt.3’ saved [10472/10472]

Samoyed 0.8846226930618286
Arctic fox 0.04580523073673248
white wolf 0.044276077300310135
Pomeranian 0.005621373653411865
Great Pyrenees 0.004652000498026609
